<a href="https://colab.research.google.com/github/takemura95/.github/blob/main/quickstarts/Get_started_OpenAI_Compatibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [5]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting started with the Gemini API OpenAI compatibility

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_OpenAI_Compatibility.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This example illustrates how to interact with the [Gemini API](https://ai.google.dev/gemini-api/docs) using the [OpenAI Python library](https://github.com/openai/openai-python).

This notebook will walk you through:

* Perform basic text generation using Gemini models via the OpenAI library
* Experiment with multimodal interactions, sending images on your prompts
* Extract information from text using structured outputs (ie. specific fields or JSON output)
* Use Gemini API tools, like function calling
* Generate embeddings using Gemini API models

More details about this OpenAI compatibility on the [documentation](https://ai.google.dev/gemini-api/docs/openai).

## Setup

### Install the required modules

While running this notebook, you will need to install the following requirements:
- The [OpenAI python library](https://pypi.org/project/openai/)
- The pdf2image and pdfminer.six (and poppler-utils as its requirement) to manipulate PDF files

In [ ]:
%pip install -U -q openai pillow pdf2image pdfminer.six
!apt -qq -y install poppler-utils # required by pdfminer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.9 MB/s eta 0:00:00
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.


## Get your Gemini API key

You will need your Gemini API key to perform the activities part of this notebook. You can generate a new one at the [Get API key](https://aistudio.google.com/app/apikey) AI Studio page.

In [ ]:
from openai import OpenAI

try:
  # if you are running the notebook on Google Colab
  # and if you have saved your API key in the
  # Colab secrets
  from google.colab import userdata

  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

except:
  # enter manually your API key here if you are not using Google Colab
  GOOGLE_API_KEY = "--enter-your-API-key-here--"

# OpenAI client
client = OpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

## Define the Gemini model to be used

You can start by listing the available models using the OpenAI library.

In [ ]:
models = client.models.list()
for model in models:
  if 'gemini-2.0' in model.id:
    print(model.id)

## Define the Gemini model to be used

In this example, you will use the `gemini-2.0-flash` model. For more details about the available models, check the [Gemini models](https://ai.google.dev/gemini-api/docs/models/gemini) page from the Gemini API documentation.

In [ ]:
MODEL="gemini-2.0-flash"

## Initial interaction - generate text

For your first request, use the OpenAI SDK to perform text generation with a text prompt.

In [ ]:
from IPython.display import Markdown

prompt = "What is generative AI?" # @param

response = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {
      "role": "user",
      "content": prompt
    }
  ]
)

Markdown(response.choices[0].message.content)

### Generating code

You can work with the Gemini API to generate code for you.

In [ ]:
prompt = """
    Write a C program that takes two IP addresses, representing the start and end of a range
    (e.g., 192.168.1.1 and 192.168.1.254), as input arguments. The program should convert this
    IP address range into the minimal set of CIDR notations that completely cover the given
    range. The output should be a comma-separated list of CIDR blocks.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

Markdown(response.choices[0].message.content)

## Multimodal interactions

Gemini models are able to process different data modatilities, such as unstructured files, images, audio and videos, allowing you to experiment with multimodal scenarios where you can ask the model to describe, explain, get insights or extract information out of those multimedia information included into your prompts. In this section you will work across different senarios with multimedia information.

**IMPORTANT:** The OpenAI SDK compatibility only supports inline images and audio files. For videos support, use the [Gemini API's Python SDK](https://ai.google.dev/gemini-api/docs/sdks).

### Working with images (a single image)

You will first download the image you want to work with.

In [ ]:
from PIL import Image as PImage


# define the image you want to download
image_url = "https://storage.googleapis.com/generativeai-downloads/images/Japanese_Bento.png" # @param
image_filename = image_url.split("/")[-1]

# download the image
!wget -q $image_url

# visualize the downloaded image
im = PImage.open(image_filename)
im.thumbnail([620,620], PImage.Resampling.LANCZOS)
im

Now you can encode the image and work with the OpenAI library to interact with the Gemini models.

In [ ]:
import base64
import requests


# define a helper function to encode the images in base64 format
def encode_image(image_path):
  image = requests.get(image_path)
  return base64.b64encode(image.content).decode('utf-8')

# Getting the base64 encoding
encoded_image = encode_image(image_url)

response = client.chat.completions.create(
  model=MODEL,
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Describe the items on this image. If there is any non-English text, translate it as well"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{encoded_image}",
          },
        },
      ],
    }
  ]
)

Markdown(response.choices[0].message.content)

### Working with images (multiple images)

You can do the same process while sending multiple images into the same prompt.

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image
from matplotlib.pyplot import imread


# define the images you want to download
image_urls = [
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/cesar-couto-OB2F6CsMva8-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/daniil-silantev-1P6AnKDw6S8-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/ruslan-bardash-4kTbAMRAHtQ-unsplash.jpg",
    "https://storage.googleapis.com/github-repo/img/gemini/retail-recommendations/furnitures/scopic-ltd-NLlWwR4d3qU-unsplash.jpg",
]

for url in image_urls:
    display(Image(url=url, width=200, height=250))

Now you can encode the images and send them with your prompt.

In [ ]:
import base64
import requests


# define a helper function to encode the images in base64 format
def encode_image(image_path):
  image = requests.get(image_path)
  return base64.b64encode(image.content).decode('utf-8')


# Getting the base64 encoding
encoded_images =[]
for image in image_urls:
  encoded_images.append(encode_image(image))

response = client.chat.completions.create(
  model=MODEL,
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Describe for what type of living room each of those items are the best match"
        },
        *[{
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_data}",
                    },
        }
                for image_data in encoded_images]
      ],
    }
  ]
)

Markdown(response.choices[0].message.content)

### Working with audio files

You can also send audio files on your prompt. Audio data provides a more rich input than text alone, and can be use for tasks like transcription, or as direct prompting like a voice assistant.

First you need to download the audio you want to use.

In [ ]:
from IPython.display import Audio


audio_url = "https://storage.googleapis.com/generativeai-downloads/data/Apollo-11_Day-01-Highlights-10s.mp3" # @param
audio_filename = audio_url.split("/")[-1]

# download the audio
!wget -q $audio_url

# listen to the downloaded audio
display(Audio(audio_filename, autoplay=False))

Now you will encode the audio in `base64` and send it as part of your request prompt.

In [ ]:
# define a helper function to encode the images in base64 format
def encode_audio(audio_path):
  with open(audio_path, 'rb') as audio_file:
    audio_content = audio_file.read()
    return base64.b64encode(audio_content).decode('utf-8')

base64_audio = encode_audio(audio_filename)

prompt = "Transcribe this audio file. After transcribing, tell me from what this can be related to." # @param
response = client.chat.completions.create(
    model=MODEL,
    messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt,
        },
        {
              "type": "input_audio",
              "input_audio": {
                "data": base64_audio,
                "format": "mp3"
          }
        }
      ],
    }
  ],
)

Markdown(response.choices[0].message.content)

## Structured outputs

Gemini API allows you to format the way your response you be generated via [structured outputs](https://ai.google.dev/gemini-api/docs/structured-output). You can define the structure you want to be used as a defined schema and, using the OpenAI library, you send this structure as the `response_format` parameter.

In this example you will:
- download a scientific paper
- extract its information
- define the structure you want your response in
- send your request using the `response_format` parameter

First you need to download the reference paper. You will use the [Attention is all your need](https://arxiv.org/pdf/1706.03762.pdf) Google paper that introduced the [Transformers architecture](https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)).

In [ ]:
from IPython.display import Image
from pdf2image import convert_from_path


# download the PDF file
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf" # @param
pdf_filename = pdf_url.split("/")[-1]
!wget -q $pdf_url

## visualize the pdf as an image
# convert the PDF file to images
images = convert_from_path(pdf_filename, 200)
for image in images:
  image.save('cover.png', "PNG")
  break

# show the pdf first page
Image('cover.png', width=500, height=600)

Now you will create your reference structure. It will be a Python `Class` that will refer to the title, authors, abstract and keywords from the paper.

In [ ]:
from pydantic import BaseModel


class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

Now you will do your request to the Gemini API sending the pdf file and the reference structure.

In [ ]:
import json
from pdfminer.high_level import extract_text


# extract text from the PDF
pdf_text = extract_text(pdf_filename)

prompt = """
    As a specialist in knowledge organization and data refinement, your task is to transform
    raw research paper content into a clearly defined structured format. I will provide you
    with the original, free-form text. Your goal is to parse this text, extract the pertinent
    information, and reconstruct it according to the structure outlined below.
"""

# send your request to the Gemini API
completion = client.beta.chat.completions.parse(
  model=MODEL,
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": pdf_text}
  ],
  response_format=ResearchPaperExtraction,
)

print(completion.choices[0].message.parsed.model_dump_json(indent=2))

Given the Gemini API ability to handle structured outputs, you can work in more complex scenarios too - like using the structured output functionality to help you generating user interfaces.

First you define the Python classes that represent the structure you want in the output.

In [ ]:
from enum import Enum


class UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    type: UIType
    label: str
    children: list[str]
    attributes: list[Attribute]

UI.model_rebuild() # This is required to enable recursive types

class Response(BaseModel):
    ui: UI

Now you send your request using the `Response` class as the `response_format`.

In [ ]:
completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a UI generation assistant. Convert the user input into a UI."},
        {"role": "user", "content": "Make a User Profile Form including all required attributes"}
    ],
    response_format=Response,
)

print(completion.choices[0].message.content)

## Developing with the Gemini API Function Calling

The Gemini API's function calling feature allows you to extend the model's capabilities by providing descriptions of external functions or APIs.

For further understanding of how function calling works with Gemini models, check the [Gemini API documentation](https://ai.google.dev/gemini-api/docs/function-calling).

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "Gets the weather at the user's location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string"},
        },
      },
    },
  }
]

Now you add the `tools` structure on your request.

In [ ]:
prompt = """
    What's the weather like in Boston?
"""

completion = client.chat.completions.create(
  model=MODEL,
  messages=[{"role": "user", "content": prompt}],
  tools=tools,
)

print(completion.choices[0].message.tool_calls[0])

## Thinking

Gemini 2.5 models are trained to think through complex problems, leading to significantly improved reasoning. The Gemini API comes with a ["thinking budget" parameter](https://ai.google.dev/gemini-api/docs/thinking) which gives fine grain control over how much the model will think.

Unlike the Gemini API, the OpenAI API offers three levels of thinking control: "low", "medium", and "high", which are mapped to 1K, 8K, and 24K thinking token budgets.

If you want to disable thinking, you can set the reasoning effort to "none".

In [ ]:
THINKING_MODEL = "gemini-2.5-flash"
prompt = """
    What is 45-78+5x13?
    Double check and explain why your answer is correct.
"""

response = client.chat.completions.create(
  model=THINKING_MODEL,
  reasoning_effort="low",
  messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {
        "role": "user",
        "content": prompt
      }
  ]
)

Markdown(response.choices[0].message.content)

## Generating and working with embeddings

Text embeddings offer a compressed, vector-based representation of text, typically in a lower-dimensional space. The core principle is that semantically similar texts will have embeddings that are spatially proximate within the embedding vector space. This representation enables solutions to several prevalent NLP challenges, including:

- **Semantic Search:** Identifying and ranking texts based on semantic relatedness.
- **Recommendation:** Suggesting items whose textual descriptions exhibit semantic similarity to a given input text.
- **Classification:** Assigning text to categories based on the semantic similarity between the text and the category's representative text.
- **Clustering:** Grouping texts into clusters based on the semantic similarity reflected in their respective embedding vectors.
- **Outlier Detection:** Identifying texts that are semantically dissimilar from the majority, as indicated by their distance in the embedding vector space.

For more details about working with the Gemini API and embeddings, check the [API documentation](https://ai.google.dev/gemini-api/docs/embeddings).

In this example you will use the `gemini-embedding-exp-03-07` model from the Gemini API to generate your embeddings.

In [ ]:
EMBEDDINGS_MODEL="gemini-embedding-exp-03-07"
prompt = """
    The quick brown fox jumps over the lazy dog.
"""

response = client.embeddings.create(
  model=EMBEDDINGS_MODEL,
  input=prompt,
)

print(len(response.data[0].embedding))
print(response.data[0].embedding[:4], '...')

A simple application of text embeddings is to calculate the similarity between sentences (ie. product reviews, documents contents, etc). First you will create a group of sentences.

In [ ]:
import pandas as pd


text = [
    "i really enjoyed the movie last night",
    "so many amazing cinematic scenes yesterday",
    "had a great time writing my Python scripts a few days ago",
    "huge sense of relief when my .py script finally ran without error",
    "O Romeo, Romeo, wherefore art thou Romeo?",
]

df = pd.DataFrame(text, columns=["text"])
df

Now you can create a function to generate embeddings, apply that function to your dataframe `text` column and save it into a new column called `embeddings`.

In [ ]:
def generate_embeddings(text):
  response = client.embeddings.create(
    model=EMBEDDINGS_MODEL,
    input=text,
  )
  return response.data[0].embedding

df["embeddings"] = df.apply(
    lambda x: generate_embeddings([x.text]), axis=1
)
df

Now that you have the embeddings representations for all sentences, you can calculate their similarities.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_array = cosine_similarity(list(df.embeddings.values))

# display as DataFrame
analysis = pd.DataFrame(cos_sim_array, index=text, columns=text)
analysis

You can also plot it for a better visualization.

In [ ]:
import seaborn as sns

ax = sns.heatmap(analysis, annot=True, cmap="Blues")
ax.xaxis.tick_top()
ax.set_xticklabels(text, rotation=90)

## Next Steps

### Do more with Gemini

If you want to use more of the Gemini capabilities and especially its unique capabilities not available through the OpenAI compatibility, you should check out the [Google GenAI SDK](https://github.com/googleapis/python-genai).

The Cookbook is full of examples on how to use it but it is recommended to start with the [Getting started](./Get_started.ipynb) notebook to get a feel of all the models and SDK capabilities.

### Related examples

Check the rest of the [Cookbook](https://github.com/google-gemini/cookbook). You'll learn how to use the [Live API](./Get_started_LiveAPI.ipynb), juggle with [multiple tools](../examples/LiveAPI_plotting_and_mapping.ipynb) or use Gemini 2.0 [spatial understanding](./Spatial_understanding.ipynb) abilities.

Also check the [Thinking cookbook](./Get_started_thinking.ipynb) that explicitly showcases its thoughts and can manage more complex reasonings.